In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**IMPORTING LIBRARIES**

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier

#load the dataset
df=pd.read_csv("/content/drive/MyDrive/Placement data analysis project dataset/Placement_Data_Full_Class.csv")
df.head()



sscb- Secondary School Certificate Board-> Which is lower secondary school(1st-10th Grade)

hscb-Higher Secondary Certificate Board-> Which is higher secondary school(11th and 12th Grade)

sl_no : serial number

gender : [male,female]

ssc_p : means--> 10th class percentage

ssc_b : which board to passed out 10th.

hsc_p : means--> 12th class percentage

hsc_b : which board to passed out 12th.

hsc_s : which stream he choose (science,commerce,arts)

degree_p : means--> Bachelor degree percentage

degree_t : which strem choose for bachelor

workex : It has a work experience or not.

etest_p : entrance test percentage

specialisation : Master degree in Mkt&HR or Mkt&Fin

mba_p : means--> Master degree percentage

status : He/She got placed or not in campus placement.

salary : placement packages.

In [ ]:
#set the sl_no as a row no
df.set_index('sl_no',inplace=True)
df.head()

In [ ]:
#check datatype or null
#prints information about dataframe
df.info()

In [ ]:
df.describe()

In [ ]:
print(df.shape[0]) # nummber of rows
print(df.shape[1]) # total number of columns

In [ ]:
df.isnull().sum()

In [ ]:
# Replace the value with 0 and 1
df["gender"].replace(["F","M"],[0,1],inplace=True)
df["status"].replace(['Placed',"Not Placed"],[1,0],inplace=True)

df['workex'].replace(to_replace ="Yes", value =1,inplace=True)
df['workex'].replace(to_replace ="No", value =0,inplace=True)
df.head()

In [ ]:
# percentage of male or female
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.axis('equal')
gender = ['Male','Female']
students = [139,76]
ax.pie(students, labels = gender, autopct='%1.2f%%', colors = ["#151436", "#b0689e"])
plt.title('Pie chart ')
plt.show()

In [ ]:
#show the no. of student how placed or not placed.
plt.bar([0],height=len(df[df["status"]==0]))
plt.bar([1],height=len(df[df["status"]==1]))

plt.xlabel("Status")
plt.ylabel("Count")

plt.xticks(np.arange(2), ('Not placed', 'Placed'))
plt.title("No of Student placed\n")
plt.show()

In [ ]:
#create a new dataframe of only numerical values
numerical_df=df.select_dtypes(["float64","int64"])

In [ ]:
#average percentage of all education qualification
values = [(numerical_df['ssc_p'].mean()),(numerical_df['hsc_p'].mean()),(numerical_df['mba_p'].mean()),(numerical_df['degree_p'].mean())]
print('scc_p mean = ' +str(numerical_df['ssc_p'].mean()))
print('hsc_p mean = ' +str(numerical_df['hsc_p'].mean()))
print('mba_p mean = ' +str(numerical_df['mba_p'].mean()))
print('degree_p mean = ' +str(numerical_df['degree_p'].mean()))
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
names = ['ssc_p','hsc_p','mba_p','degree_p']
ax.set_ylabel('Average percentages')
ax.set_title('Average Percentage')
ax.bar(names,values,width = 0.5,color=["#2ca02c"])
plt.show()

**PLACEMENT PREDICTION**

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(df[['ssc_p','hsc_p','degree_p','workex','etest_p','mba_p']],df.status,test_size=0.2)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)              # determine parameters and transform dataset
x_test = sc.transform(x_test)                    # transform data for further analysis


**RANDOM FOREST CLASSIFICATION**

In [ ]:
# Fitting Random Forest Classification to the Training set
accuracy_scores = np.zeros(4)
def rfc_model(x_train,y_train,x_test,y_test):

    '''
    Input
      x_train :training featue
      x_test  :test featue
      y_train :training output data
      y_test  :testing  output data

    Output :
     accuracy_score       :
     classification_report:
    '''
    print("using Random Forest Classification method")
    model=RandomForestClassifier(n_estimators =14, criterion = 'entropy', random_state = 41)
    model.fit(x_train,y_train)
    prediction=model.predict(x_test)
    print('Accuracy {0:.2f}%'.format((accuracy_score(y_test,prediction)*100)))
    print(classification_report(y_test,prediction))
    accuracy_scores[0] = accuracy_score(y_test, prediction)*100
rfc_model(x_train,y_train,x_test,y_test)
print('Randon Forest Classifier accuracy: {}%'.format(accuracy_scores[0]))

**K NEAREST NEIGHBOUR**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
result_1 = pd.DataFrame(columns=['Model','Accuracy Score'])
#K Nearest Neighbours Classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
weights = ['uniform','distance']
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
metric = ['minkowski','euclidean','manhattan']
algorithm=['auto']
p=[1,2]
knn_hyperparameters = dict(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm, leaf_size=leaf_size, p=p, metric=metric)
KNN_gridSearch = RandomizedSearchCV(knn,param_distributions=knn_hyperparameters,cv=3,verbose=1)
KNN_gridSearch.fit(x_train,y_train)
knn_y_pred = KNN_gridSearch.predict(x_test)
print('KNN Classifier')
print('\n Confusion Matrix')
print(confusion_matrix(y_test,knn_y_pred))
print('\n Classification Report')
print(classification_report(y_test,knn_y_pred))
knn_AS = accuracy_score(y_test,knn_y_pred)

accuracy_scores[1] = accuracy_score(y_test,knn_y_pred)*100
print('K Nearest Neighbors Classifier accuracy: {}%'.format(accuracy_scores[1]))

**SVM**

In [ ]:
from sklearn.svm import SVC
svc = SVC()
C = [0.1, 1, 10, 100, 1000]
kernel = ['rbf','poly','sigmoid','linear']
degree = list(range(1,5,1))
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
svc_hyperparameters = dict(C=C, kernel=kernel, degree=degree, gamma=gamma)
svc_gridSearch = RandomizedSearchCV(svc,param_distributions=svc_hyperparameters,cv=3,verbose=1)
svc_gridSearch.fit(x_train,y_train)  # grid search simplest algo for hyperparameter tuning
svc_y_pred = svc_gridSearch.predict(x_test)
print('Support Vector Machine (SVC)')
print('\n Confusion Matrix')
print(confusion_matrix(y_test,svc_y_pred))
print('\n Classification Report')
print(classification_report(y_test,svc_y_pred))
svc_AS = accuracy_score(y_test,svc_y_pred)
accuracy_scores[2] = accuracy_score(y_test,svc_y_pred)*100
print('SVM Classifier accuracy: {}%'.format(accuracy_scores[2]))

**DECISION TREE**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
criterion=['gini','entropy']
splitter=['best', 'random']
max_depth = list(range(10,100,10))
min_samples_split = [2, 5, 10]
max_features = ['auto', 'sqrt','log2',None]
max_depth = list(range(10,100,10))
min_samples_leaf = [1, 2, 4]
dtc_hyperparameters = dict(criterion=criterion, splitter=splitter, max_depth=max_depth, min_samples_split=min_samples_split, max_features=max_features, min_samples_leaf=min_samples_leaf)
dtc_gridSearch = RandomizedSearchCV(dtc,param_distributions=dtc_hyperparameters,cv=3,verbose=1)
dtc_gridSearch.fit(x_train,y_train)
dtc_y_pred = dtc_gridSearch.predict(x_test)
print('Decision Tree Classifier')
print('\n Confusion Matrix')
print(confusion_matrix(y_test,dtc_y_pred))
print('\n Classification Report')
print(classification_report(y_test,dtc_y_pred))
dtc_AS = accuracy_score(y_test,dtc_y_pred)
accuracy_scores[3] = accuracy_score(y_test,dtc_y_pred)*100
print('decision tree Classifier accuracy: {}%'.format(accuracy_scores[3]))
